In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import read_and_prepare_data

In [2]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [2]:
trafo_id = "T1330"
data = read_and_prepare_data(trafo_id)

date_time                        0
SMM                              0
trafo_station_id                 0
active_power                     0
reactive_power                   0
trafo_id                    781322
voltage                          0
temperature_2m                   0
snow_depth                       0
cloud_cover                      0
is_day                           0
shortwave_radiation              0
direct_radiation                 0
diffuse_radiation                0
direct_normal_irradiance         0
dtype: int64
date_time                   0
trafo_id                    0
trafo_station_id            0
active_power                0
reactive_power              0
voltage                     0
temperature_2m              0
snow_depth                  0
cloud_cover                 0
is_day                      0
shortwave_radiation         0
direct_radiation            0
diffuse_radiation           0
direct_normal_irradiance    0
dtype: int64
date_time                   0

In [4]:
df_edges = data["edges_static_data"]
df_measurments = data["measurements"]

In [5]:
df_measurments.isna().sum()

date_time                   0
node_id                     0
voltage                     0
temperature_2m              0
snow_depth                  0
cloud_cover                 0
is_day                      0
shortwave_radiation         0
direct_radiation            0
diffuse_radiation           0
direct_normal_irradiance    0
active_power                0
reactive_power              0
year                        0
month                       0
day                         0
hour                        0
minute                      0
dtype: int64

In [ ]:
df_measurments[df_measurments["node_id"]==0].isna().sum()

date_time                   0
node_id                     0
voltage                     8
temperature_2m              0
snow_depth                  0
cloud_cover                 0
is_day                      0
shortwave_radiation         0
direct_radiation            0
diffuse_radiation           0
direct_normal_irradiance    0
active_power                0
reactive_power              0
year                        0
month                       0
day                         0
hour                        0
minute                      0
dtype: int64

In [7]:
def get_array_of_timestemps(df_measurments):
    """
    Returns list of dfs ordered by date_time.
    """
    df_grouped = df_measurments.groupby("date_time")
    dfs = [(date, df) for date, df in df_grouped]
    dfs = sorted(dfs, key=lambda x: x[0])
    dfs = np.dstack([df.sort_values(by="node_id").drop(columns=["date_time", "node_id"]).values.T for _, df in dfs])
    return dfs

In [8]:
get_array_of_timestemps(df_measurments).shape

NameError: name 'df_measurments' is not defined

In [ ]:
data.keys()

dict_keys(['edges_static_data', 'measurements', 'measurments'])

In [10]:
class SimpleGraphVoltDatasetLoader(object):
    """
    Check this https://pytorch-geometric-temporal.readthedocs.io/en/latest/_modules/torch_geometric_temporal/dataset/wikimath.html#WikiMathsDatasetLoader
    for an example of how to implement a dataset loader

    And here are the docs https://pytorch-geometric-temporal.readthedocs.io/en/latest/modules/signal.html
    """
    def __init__(self, trafo_id):
        self._trafo_id = trafo_id
        self._read_data()

    def _read_data(self):
        dataset = read_and_prepare_data(self._trafo_id)
        self._df_edges = dataset["edges_static_data"]
        self._df_measurments = dataset["measurements"]
        self._periods = len(self._df_measurments["date_time"].unique())
        self._node_counts = len(self._df_measurments["node_id"].unique())

    def _get_edges_and_edge_weights(self):
        self._edges = self._df_edges[["from_node_id", "to_node_id"]].to_numpy().T
        self._edge_weights = self._df_edges.drop(["from_node_id", "to_node_id"], axis=1).to_numpy().T

    def _get_targets_and_features(self):
        dfs = get_array_of_timestemps(self._df_measurments) 
        self.targets = []
        self.features = []
        for i in range(self._periods-self.num_timesteps_in-self.num_timesteps_out+1):
            self.features.append(dfs[:, :, i:i+self.num_timesteps_in])
            self.targets.append(dfs[:, :, i+self.num_timesteps_in:i+self.num_timesteps_in+self.num_timesteps_out])

    def get_dataset(self, num_timesteps_in: int = 12, num_timesteps_out: int = 4) -> StaticGraphTemporalSignal:
        self.num_timesteps_in = num_timesteps_in
        self.num_timesteps_out = num_timesteps_out
        self._get_edges_and_edge_weights()
        self._get_targets_and_features()
        dataset = StaticGraphTemporalSignal(
            self._edges, 
            self._edge_weights, 
            self.features, 
            self.targets
            )
        return dataset
        

In [11]:
trafo_id = "T1330"
loader = SimpleGraphVoltDatasetLoader(trafo_id)

In [13]:
loader_data = loader.get_dataset()

In [15]:
[i.shape for i in loader.features]

[(16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 113, 12),
 (16, 11

In [16]:
[i.shape for i in loader.targets]

[(16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113, 4),
 (16, 113,